# Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../src')
import utils.utils as utils

!pip freeze > ../requirements.txt

In [2]:
raw = pd.read_csv(r'..\data\backup\raw_set.csv').drop(['SK_ID_CURR.1', 'RN', 'SK_ID_CURR.2', 'RN.1'], axis=1)
# # Logging data types
# types = pd.DataFrame(raw.dtype)
# types.to_csv('../log/dtypes.csv')
# ddesc = raw.describe().T
# ddesc.to_csv('../log/ddesc.csv')
# import pandas as pd
# dobject = []
# for col in raw.columns:
#     if raw[col].dtype == 'object' or raw[col].dtype == 'category':
#         dobject.append([col, raw[col].dtype, raw[col].nunique(), raw[col].unique()])

# dobject_df = pd.DataFrame(dobject, columns=['column', 'dtype', 'nunique', 'unique_values'])
# dobject_df.to_csv('../log/dobject.csv', index=False)

## Clean object data

In [3]:
def auto_transform_object(df, max_unique=10, output_path = "."):
    object_col = []
    log_binary_bin = []
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            object_col.append(col)
    for col in object_col:
        nunique = df[col].nunique()
        if nunique == 2:
            vals = list(df[col].unique())
            df[col] = np.where(df[col] == vals[0], 1, np.where(df[col] == vals[1], 0, np.nan))
            log_binary_bin.append([col, vals[0], vals[1]])
        elif nunique > 2 and nunique < max_unique:
            df[col] = df[col].astype('category')
        elif nunique > max_unique:
            print(f'{col} has more than {max_unique} unique values, bypass auto transform.')
    pd.DataFrame(log_binary_bin, columns=['column', 'bin_1', 'bin_0']).to_csv(output_path, index=False)

auto_transform_object(raw, max_unique=10, output_path = '../log/log_binary_bin.csv')

OCCUPATION_TYPE has more than 10 unique values, bypass auto transform.
ORGANIZATION_TYPE has more than 10 unique values, bypass auto transform.


In [4]:
raw['CODE_GENDER'] = np.where(raw['CODE_GENDER'] == 'XNA', np.nan, raw['CODE_GENDER'])
raw['CODE_GENDER'] = (raw['CODE_GENDER'] == 'M').astype(int)

raw['NAME_INCOME_TYPE'] = raw['NAME_INCOME_TYPE'].apply(lambda x: 'Others' if x in ['Unemployed', 'Student', 'Businessman', 'Maternity leave'] else x)
raw['NAME_INCOME_TYPE'] = raw['NAME_INCOME_TYPE'].astype('category')

raw['IS_UNACCOMPANIED'] = np.where(raw['NAME_TYPE_SUITE'] == 'Unaccompanied', 1, np.where(raw['NAME_TYPE_SUITE'].isna(), np.nan, 0))

raw['IS_SINGLE'] = np.where(raw['NAME_FAMILY_STATUS'] == 'Single / not married', 1, np.where(raw['NAME_FAMILY_STATUS'] == 'Unknown', np.nan, 0))

raw['OCCUPATION_TYPE'] = raw['OCCUPATION_TYPE'].astype('category')

def category(val):
    if 'Industry' in val: return 'industry'
    if 'Trade' in val: return 'trade'
    if 'Transport' in val: return 'transport'
    if 'Business Entity' in val: return 'business'
    if val in ['XNA']: return 'unknown'
    if val in ['Self-employed']: return 'self-employed'
    if val in ['Other']: return 'others_1'
    else : return 'others_2'
    return val
raw['ORGANIZATION_TYPE'] = raw['ORGANIZATION_TYPE'].apply(category)
raw['ORGANIZATION_TYPE'] = raw['ORGANIZATION_TYPE'].astype('category')

## Clean numeric data

In [5]:
raw['DAYS_EMPLOYED'] = np.where(raw['DAYS_EMPLOYED'] == '365243', np.nan, np.where(raw['DAYS_EMPLOYED'].isna(), np.nan, raw['DAYS_EMPLOYED']))

raw['OWN_CAR_OVER_40Y'] = np.where(raw['OWN_CAR_AGE'] >= 40, 1, 0)
raw['OWN_CAR_AGE'] = np.where(raw['OWN_CAR_AGE'] >= 40, 40, np.where(raw['OWN_CAR_AGE'].isna(), np.nan, raw['OWN_CAR_AGE']))

raw['AMT_INCOME_TOTAL'] = np.where(raw['AMT_INCOME_TOTAL'] == 117000000, 202500, raw['AMT_INCOME_TOTAL'])

In [6]:
raw.to_csv('../data/cleaned_data.csv', index=False)